# Reacher World!

This is the report for the Udacity Deep Reinforcement Learning Nanodegree continuous control project. See the `README.md` for a description of the Unity environment.

## Learning Algorithm

The algorithm used to solve the environment is PPO, as described in the [paper](https://arxiv.org/abs/1707.06347) and the Udacity lectures about clipped surrogate function and actor-critic methods.

Other resources consulted for the implementation are the Hugging Face [Deep RL Course](https://huggingface.co/learn/deep-rl-course) and the [CleanRL](https://github.com/vwxyzjn/cleanrl) implementation (with its wonderful [detailed explanation](https://iclr-blog-track.github.io/2022/03/25/ppo-implementation-details/)).

This implementation is notably different in various aspects and code structure (described below).

### Actor and Critic Networks

Both actor and critic networks are feed-forward neural networks with three fully-connected layers. The input size is the state size (i.e., `33` for the Reacher environment), and the hidden layer dimension is `32`. All activations are `tanh`, and the output layers are linear (i.e., there is no non-linearity processing the last linear layer output).

The critic output size is `1`, which represents the value estimate for the state(s) processed by the network. The actor output size is the action _continous space_ number of dimensions (i.e., `4` for the Reacher environment) and it represents the _mean_ value of the Gaussian distribution used to sample the continous action. Additionally for the actor, we train the standard deviation _Parameter_ for the such Gaussian distribution.

The two networks do not share layers.

#### Initialization

The neural network linear layers initialization turns out to be **critical** in order to be able to learn. In particular, weights are scaled by a factor of `1e-3` (similar to the [DDPG pendulum](https://github.com/udacity/deep-reinforcement-learning/blob/master/ddpg-pendulum/model.py) model). Without such scaling, the agent doesn't learn correctly.

It is unclear if state / action / reward normalization would help and generalize, or whether other architectures do not suffer from such initialization sensitivity. See future improvements for some consideration.

### PPO Implementation Details

#### Loss Function

The PPO loss as described in the paper is:

\begin{equation}
L_t^{CLIP + VF + S}(\theta) = \^{\mathbb{E}_t}[L_t^{CLIP}(\theta) + c_1 L_t^{VF}(\theta) + c_2 S[\pi_\theta](s_t)]
\end{equation}

Which is a combined: policy loss + value loss + entropy loss. It is important to optimize a single loss function when the network share layers and weights.

But in this implementation, actor and critic networks are separate, so I opted to optimize the two loss functions $L_t^{CLIP}(\theta)$ and $L_t^{VF}(\theta)$ separately (no entropy added), much more similarly to the actor-critic network optimization of the Udacity lesson and examples.

Clipping policy updates (and gradient) happens as described in the PPO lesson (and the paper itself).

#### Training Loop

As wonderfully explained in the [CleanRL implementation details](https://iclr-blog-track.github.io/2022/03/25/ppo-implementation-details/) and the PPO paper itself, the PPO training loop is not based on episodes / max_steps. Instead, we keep collecting _trajectory segments_ and progressively train the agent on those. That enables to train in long-lasting environments as well.

The PPO training loop has been structured accordingly. But we exploit the Reacher environment specificities (i.e., constant `1001` steps episode duration for all bots) to be able to "even out" the trajectory segments (a.k.a., policy rollout steps) and determine if the environment is solved across `100` episodes.

See the `unity_env/README.md` for more details about the Reacher environment, and the code itself for the specifics.

Finally, the learning happens in mini-batches on every trajectory segment. Having an appropriate number of mini-batches is also very important for the algorithm to learn well and fast.

#### Generalized Advantage Estimation

GAE has been used and implemented according to the [paper](https://arxiv.org/abs/1506.02438). We reset the computation if the episode ends mid-way during a trajectory segment.

Also, the standard advantage estimation has been implemented: $A_t(s, a) = Q_t(s, a) - V_t(s)$. But training takes notably longer and it is slower to learn, though it keeps learning nonetheless.

### Hyperparameters

Hyperparameters have mostly standard values:
TODO: Alessio list them!

## Training

Training runs for 125 steps, after which it is considered solved because the average score for 100 episodes (across bots) is greater than `30.0`.

Let's import the necessary module to run the training:

In [1]:
from agent import Agent
from reacher import ReacherWorld
from ppo import PPO

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [2]:
# Utility plotting function
def plot(scores):
    """Plot scores and their running average."""
    avgs = pd.Series(scores).rolling(100).mean()
    x = np.arange(len(scores))
    plt.figure('Episode scores')
    plt.plot(x, scores, label='Scores')
    plt.plot(x, avgs, 'r', label='Running average')
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

In [3]:
# Create the Reacher world!
reacher_world = ReacherWorld()

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Let's train an agent!

In [ ]:
agent = reacher_world.new_agent()
scores = reacher_world.train(agent)

In [ ]:
plot(scores)

## Future Improvements

Here are some ideas for future improvements on this project:

 * Neural network initialization: figure out if different architectures are not prone to sensitivity to initial weights.
 * Neural network sharing layers: share layers between actor and critic, and use PPO loss function.
 * Try this PPO implementation in different environments, and generalize it to accept different networks.
 * Explore different ways of determining rewards, e.g., include [_curiosity_](https://pathak22.github.io/large-scale-curiosity/).

## Appendix: Comparisons and Variations

Let's compare what happens with different hyperparameters... it is fun to see the struggle to get this right :) Also, just 50 episodes to not take forever!

### Standard Returns Computation vs. GAE

With standard return computation we can see that learning is much slower.

In [ ]:
agent = reacher_world.new_agent()
scores = reacher_world.train(agent, max_episodes=50, gae_enabled=False)
plot(scores)

### Network Weights Standard Initialization

Not rescaling linear layers weight seems to prevent learning.

In [ ]:
agent = reacher_world.new_agent(weight_mul=1.0)
scores = reacher_world.train(agent, max_episodes=50)
plot(scores)

### Train A Single Batch

Training across the entire segment (instead of mini-batches) incredibly slows down policy learning. This is to be expected b/c we perform a single update to the policy with all the collected data, instead of iterating and performing many smaller stochastic updates.

In [ ]:
agent = reacher_world.new_agent()
scores = reacher_world.train(agent, max_episodes=50, n_mini_batches=1)
plot(scores)